# Capstone Project

# Libraries

In [1]:
# Import Library 

import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from pprint import pprint
import numpy as np
from numpy import array
import pandas as pd 
import csv
import pickle
from IPython.display import display
import pickle
from collections import Counter
pd.set_option('display.max_columns', 100)
from IPython.core import display as ICD


# Define Text Normalization Function

In [2]:
def restore_windows_1252_characters(restore_string):
    """
    Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.
    """

def to_windows_1252(match):
    try:
        return bytes([ord(match.group(0))]).decode('windows-1252')
    except UnicodeDecodeError:
        # No character at the corresponding code point: remove it.
        return ''

    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [3]:
def grab_doc_content( brand, CIK ):
    
    company = {}
    
    company['auto'] = {}
    auto = {brand : CIK
           }
    company['auto'] = auto 

    key = list(auto.copy().values())
    
    return key

In [4]:
grab_doc_content('Facebook', '0001326801')

['0001326801']

In [5]:
master_filings_dict = {}
    
file_code = {}
    
file_text = {}

Accession_Number_URL = {}

In [6]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in grab_doc_content('Facebook', '0001326801')]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in grab_doc_content('Facebook', '0001326801'):

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)


In [7]:
def url_maker(dir_url):
    dir_url_list = [dir_url.format(x) for x in grab_doc_content('Facebook', '0001326801')]

    #print('Directory URL: {}'.format(dir_url_list))
    doc_url_list = [] 

    # FOR-loop yielding Accession Numbers from CIK/URL Directory.
    for CIK_num in grab_doc_content('Facebook', '0001326801'):

        doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
        doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
        
        doc_url_list.append(doc_url_new)
        
        url_lists = zip( dir_url_list , doc_url_list )
        
    return url_lists

In [8]:
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
for dir_url, doc_url in url_maker(url): 
    print(1, dir_url)
    print(2, doc_url)

1 https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001326801&type=10-&dateb=&owner=include&count=100
2 https://www.sec.gov/Archives/edgar/data/0001326801/{xx}/{yy}.txt


In [9]:
Accession_Number_URL = {}
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

for dir_url, doc_url in url_maker(url):
    
    response = requests.get(dir_url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [ cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', dir_url) ][0]
    CIKK = {cikk : accessionnumber}
    
    for (a,b) in accession_numbers: 
        master_filings_dict[b] = {}
        master_filings_dict[b]['sec_header_content'] = {}
        master_filings_dict[b]['filing_documents'] = None

        doc_url_single = doc_url.format(xx = a, yy = b)

        file_url_list = []

        

        file_url_list.append( doc_url_single )
        
        Accession_Number_URL.update({ b : file_url_list})

In [10]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0001326801-14-000068':
        break
        
    else:
        A_N.append(AN)

print(A_N)
print(len(A_N))

['0001326801-20-000048', '0001326801-20-000013', '0001326801-19-000069', '0001326801-19-000055', '0001326801-19-000037', '0001326801-19-000009', '0001326801-18-000067', '0001326801-18-000057', '0001326801-18-000032', '0001326801-18-000009', '0001326801-17-000053', '0001326801-17-000038', '0001326801-17-000024', '0001326801-17-000007', '0001326801-16-000087', '0001326801-16-000082', '0001326801-16-000067', '0001326801-16-000063', '0001326801-16-000043', '0001326801-15-000032', '0001326801-15-000028', '0001326801-15-000015', '0001326801-15-000010', '0001326801-15-000006']
24


## Save each Filing.

In [11]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0001326801-14-000068':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('hr')

        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks]

        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 81 page(s) found.
There was 81 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001326801-20-000048 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 108 page(s) found.
There was 108 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001326801-20-000013 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 71 page(s) found.
There was 71 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001326801-19-000069 were parsed and stored.

In [ ]:
string_test = master_filings_dict['0001326801-20-000048']['filing_documents']['10-Q']['pages_code']
string_test

## Anchor Check Function



In [12]:
# Variations of HTML anchors.
CBS_A = 'CONDENSED CONSOLIDATED BALANCE SHEETS'
CBS_B = 'CONSOLIDATED BALANCE SHEETS'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


In [13]:
# TESTING: CONDENSED CONSOLIDATED BALANCE SHEETS
for k, v in QorK.items():
        
    for i, vv in enumerate(v):
        print('\n', i, vv)
        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code'] 


        for page in pages:

            if  anchor_check(page) and ('In millions' in page):
                print('\n', 'Pass Anchor check')

                # Convert string to BS object
                soup = BeautifulSoup(page, 'html5')

                # then get all the rows in the table.
                table_rows = soup.find_all('tr')

                if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
                    del table_rows[0]
                    print('another table row deleted', '\n')
                single_table = []

                # Loop through each column, adding in three structures to single_table.
                for tr_post, tr in enumerate(table_rows):

                    td = tr.find_all('td')

                    column = [tr.text for tr in td]
                    #print( len(column), column)
                    if tr_post == 3:

                        col_date = Quarter( column )
                        single_table.append( col_date )
                        
                        col_year = year( column )
                        single_table.append( col_year )
                    
                    elif tr_post > 5: 
                        
                        cleaned_data = cleaning_column( column , 10**6 )

                        if len(cleaned_data) > 10:
                            del cleaned_data[3]

                        single_table.append( cleaned_data )
                
                
                print(single_table)
            
            


 0 0001326801-20-000048

 Pass Anchor check
another table row deleted 



NameError: name 'Quarter' is not defined

## Cleaning Columns Function


In [14]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [15]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

### Cleaning Functions.

In [16]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

In [18]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        try:
            ele= element.encode('cp1252').decode("utf-8", "ignore")
                
        except:
            ele= element.encode('latin1').decode("utf-8", "ignore")
            
        if ele =='':
            continue
            
        ele_norm = unicodedata.normalize('NFKD', ele)
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif ele_norm not in ['\n$', '\n\xa0', '\n', '\xa0']:
            
            n_element = ele_norm.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)
            
            result = [int(clean_ele) * m for clean_ele in res]

            column_list.extend(result)

    return column_list
    

# Throw column list-like object into cleaning function. 
def cleaning_column(column, m):
     
    column_list = []

    for element_post, element in enumerate(column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:

            element = unicodedata.normalize('NFKD', element)
            
            element = element.replace( '\n', '')
            
            clean_ele = clean_col_name( element )
            
            column_list.append(clean_ele)
            
        # Use a dictionary to convert the Months into numbers.
        #elif element_post == :
            #month = re.sub('({})'.format('|'.join(map(re.escape, month_dict.keys()))), lambda m: month_dict[m.group()], dat)
            #column_list.extend(month)
        #elif element == '(' or ')': 
            #pint(element)
            #continue
            
        else:
            
            element = element.replace( '\n$' and '\n\xa0' and '\n', '0')
            
            element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(element)

            res = [int(ele) * m for ele in res]

            column_list.extend(res)

    return column_list
    

## Cleaning Tables Function

In [19]:
def Quarter( column ):
    
    Q = []
    
    for ele_num, ele in enumerate(column):
        
        if ele != '\xa0':
            
            date_data = re.sub('({})'.format('|'.join(map(re.escape, month_dict.keys()))), lambda m: month_dict[m.group()], ele)

            pattern = re.compile(r'\w+')

            res = pattern.findall(date_data)
            #print('Qres', '-' , res)

            Q.append(res[0])

    Q.insert(0, 'Quarter')
    
    return Q

In [20]:
def year( column ):
    
    years = []
    
    for ele_num, ele in enumerate(column):
        
        if ele != '\xa0':
            
            clean_year = re.sub('({})'.format('|'.join(map(re.escape, month_dict.keys()))), lambda m: month_dict[m.group()], ele)

            pattern = re.compile(r'\w+')

            res = pattern.findall(clean_year)
            #print('Yres', '-' , res)
            try:
                years.append(res[2])
            
            except:
                years.append(res[0])
                
    years.insert(0, 'Year')
    
    return years

In [ ]:
['\xa0', 'As of December 31, 2015', '\xa0', 'As ofSeptember 30, 2016']

In [21]:
def table_extractor( page, multiplier, k ):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
    
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]

        # 10-Q 
        if k == '10-Q':
            
            if tr_post == 3:

                # Quarter Column
                col_Q = Quarter( column )
                single_table.append( col_Q )

                # Years Column
                col_year = year( column )
                single_table.append( col_year )

            elif tr_post > 5: 

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
                
            if tr_post == 1:
                
                # Quarter Column
                col_date = Quarter( column )
                single_table.append( col_date )
            
            elif tr_post == 2:
                
                col_year = year( column )
                single_table.append( col_year )

            elif tr_post > 4: 
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [22]:
def df_clean( single_table ):
    #print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  

    #table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [23]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if anchor_check(page) & ('In millions' in page):
            
            single_table = table_extractor( page, 10**6, k )
            
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif anchor_check(page) & ('In thousands' in page):
            
            single_table = table_extractor( page, 10**3, k )
            
            table_df = df_clean(single_table)
            
            doc_tables.append(table_df)
                
    return doc_tables
    

# ***************Making this only to rectify 10-Ks in Facebook***************

In [31]:
explore_10_K =  {'10-K': ['0001326801-20-000013',
  '0001326801-19-000009',
  '0001326801-18-000009',
  '0001326801-17-000007',
  '0001326801-16-000043',
  '0001326801-15-000006']}

In [32]:
for QK, N_O in explore_10_K.items():
    
    for an_postition, acc_num in enumerate(N_O): 
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df = table(pages, QK)
        
        # Hereinafter is executed in part 2.
        table_df = test_df[0].drop([1]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)


 This is 0001326801-20-000013 a 10-K document.
Shape -  (1, 33)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,None,2018,10019000000,31095000000,7587000000,1779000000,50480000000,24683000000,None,1294000000,18301000000,2576000000,97334000000,None,None,None,820000000,541000000,None,5509000000,147000000,7017000000,None,6190000000,13207000000,None,None,None,42906000000,760000000,41981000000,84127000000,97334000000



 This is 0001326801-19-000009 a 10-K document.
Shape -  (1, 30)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,None,2017,8079000000,33632000000,5832000000,1020000000,48563000000,13721000000,1884000000,18221000000,2135000000,84524000000,None,None,None,380000000,390000000,2892000000,98000000,3760000000,6417000000,10177000000,None,None,None,40584000000,227000000,33990000000,74347000000,84524000000



 This is 0001326801-18-000009 a 10-K document.
Shape -  (1, 30)


...

,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,None,2015,4907000000,13527000000,2559000000,659000000,21652000000,5687000000,3246000000,18026000000,796000000,49407000000,None,None,None,196000000,217000000,1449000000,56000000,None,1925000000,None,3157000000,5189000000,None,None,None,34886000000,455000000,9787000000,44218000000,49407000000



 This is 0001326801-16-000043 a 10-K document.
Shape -  (1, 32)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,None,2014,4315000000,6884000000,1678000000,513000000,13390000000,3967000000,3929000000,17981000000,699000000,39966000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2327000000,3870000000,None,None,None,30225000000,228000000,6099000000,36096000000,39966000000



 This is 0001326801-15-000006 a 10-K document.
Shape -  (1, 32)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,None,2013,3323000000,8126000000,1109000000,512000000,13070000000,2882000000,883000000,839000000,221000000,17895000000,None,None,None,87000000,181000000,555000000,38000000,239000000,1100000000,237000000,1088000000,2425000000,None,None,None,12297000000,14000000,3159000000,15470000000,17895000000


# ***************Special Save for 10-Ks***************

In [34]:
Checklist = {}

for k, v in explore_10_K.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            table_df = table(pages, k)                 
            ICD.display(table_df[0])

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2019,19079000000,35776000000,9518000000,1852000000,66225000000,35323000000,9460000000,894000000,18715000000,2759000000,133376000000,None,None,None,1363000000,886000000,800000000,11735000000,269000000,15053000000,9524000000,7745000000,32322000000,None,None,None,45851000000,489000000,55692000000,101054000000,133376000000
2,None,2018,10019000000,31095000000,7587000000,1779000000,50480000000,24683000000,None,1294000000,18301000000,2576000000,97334000000,None,None,None,820000000,541000000,None,5509000000,147000000,7017000000,None,6190000000,13207000000,None,None,None,42906000000,760000000,41981000000,84127000000,97334000000


All the pages from 10-K document 0001326801-20-000013 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2018,10019000000,31095000000,7587000000,1779000000,50480000000,24683000000,1294000000,18301000000,2576000000,97334000000,None,None,None,820000000,541000000,5509000000,147000000,7017000000,6190000000,13207000000,None,None,None,42906000000,760000000,41981000000,84127000000,97334000000
2,None,2017,8079000000,33632000000,5832000000,1020000000,48563000000,13721000000,1884000000,18221000000,2135000000,84524000000,None,None,None,380000000,390000000,2892000000,98000000,3760000000,6417000000,10177000000,None,None,None,40584000000,227000000,33990000000,74347000000,84524000000


All the pages from 10-K document 0001326801-19-000009 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2017,8079000000,33632000000,5832000000,1020000000,48563000000,13721000000,1884000000,18221000000,2135000000,84524000000,None,None,None,380000000,390000000,2892000000,98000000,3760000000,6417000000,10177000000,None,None,None,40584000000,227000000,33990000000,74347000000,84524000000
2,None,2016,8903000000,20546000000,3993000000,959000000,34401000000,8591000000,2535000000,18122000000,1312000000,64961000000,None,None,None,302000000,280000000,2203000000,90000000,2875000000,2892000000,5767000000,None,None,None,38227000000,703000000,21670000000,59194000000,64961000000


...

All the pages from 10-K document 0001326801-17-000007 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2015,4907000000,13527000000,2559000000,659000000,21652000000,5687000000,3246000000,18026000000,796000000,49407000000,None,None,None,196000000,217000000,1449000000,56000000,7000000,1925000000,107000000,3157000000,5189000000,None,None,None,34886000000,455000000,9787000000,44218000000,49407000000
2,None,2014,4315000000,6884000000,1678000000,513000000,13390000000,3967000000,3929000000,17981000000,699000000,39966000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2327000000,3870000000,None,None,None,30225000000,228000000,6099000000,36096000000,39966000000


All the pages from 10-K document 0001326801-16-000043 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2014,4315000000,6884000000,1678000000,793000000,13670000000,3967000000,3929000000,17981000000,637000000,40184000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2545000000,4088000000,None,None,None,30225000000,228000000,6099000000,36096000000,40184000000
2,None,2013,3323000000,8126000000,1109000000,512000000,13070000000,2882000000,883000000,839000000,221000000,17895000000,None,None,None,87000000,181000000,555000000,38000000,239000000,1100000000,237000000,1088000000,2425000000,None,None,None,12297000000,14000000,3159000000,15470000000,17895000000


All the pages from 10-K document 0001326801-15-000006 have been tableized and saved.
--------------------------------------------------------------------------------


# ***************10-K special ends***************

# Exploring the Data

In [25]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [26]:
explore_dict

{'10-Q': ['0001326801-20-000048', '0001326801-15-000015'],
 '10-K': ['0001326801-20-000013', '0001326801-15-000006']}

In [27]:
for QK, N_O in explore_dict.items():
    
    for an_postition, acc_num in enumerate(N_O): 
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df = table(pages, QK)
        
        # Hereinafter is executed in part 2.
        table_df = test_df[0].drop([2]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)


 This is 0001326801-20-000048 a 10-Q document.
Shape -  (1, 33)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2020,23618000000,36671000000,7289000000,1771000000,69349000000,37127000000,9359000000,838000000,18811000000,2887000000,138371000000,None,None,None,829000000,712000000,835000000,12446000000,247000000,15069000000,9509000000,8489000000,33067000000,None,None,None,46688000000,544000000,59160000000,105304000000,138371000000



 This is 0001326801-15-000015 a 10-Q document.
Shape -  (1, 30)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q1,2015,3419000000,8994000000,1508000000,1083000000,15004000000,4619000000,3774000000,18005000000,605000000,42007000000,None,None,129000000,189000000,1300000000,55000000,73000000,1746000000,113000000,2647000000,4506000000,None,None,31337000000,447000000,6611000000,37501000000,42007000000



 This is 0001326801-20-000013 a 10-K document.
Shape -  (1, 33)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2019,19079000000,35776000000,9518000000,1852000000,66225000000,35323000000,9460000000,894000000,18715000000,2759000000,133376000000,None,None,None,1363000000,886000000,800000000,11735000000,269000000,15053000000,9524000000,7745000000,32322000000,None,None,None,45851000000,489000000,55692000000,101054000000,133376000000



 This is 0001326801-15-000006 a 10-K document.
Shape -  (1, 32)


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive,Retained Earnings,Shareholder Equity,Total liabilities and equity
0,Q4,2014,4315000000,6884000000,1678000000,793000000,13670000000,3967000000,3929000000,17981000000,637000000,40184000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2545000000,4088000000,None,None,None,30225000000,228000000,6099000000,36096000000,40184000000


In [28]:
QorK

{'10-Q': ['0001326801-20-000048',
  '0001326801-19-000069',
  '0001326801-19-000055',
  '0001326801-19-000037',
  '0001326801-18-000067',
  '0001326801-18-000057',
  '0001326801-18-000032',
  '0001326801-17-000053',
  '0001326801-17-000038',
  '0001326801-17-000024',
  '0001326801-16-000087',
  '0001326801-16-000082',
  '0001326801-16-000067',
  '0001326801-15-000032',
  '0001326801-15-000028',
  '0001326801-15-000015'],
 '10-K': ['0001326801-20-000013',
  '0001326801-19-000009',
  '0001326801-18-000009',
  '0001326801-17-000007',
  '0001326801-16-000043',
  '0001326801-15-000006']}

# Save Files Locally. 

In [29]:
for k, v in QorK.items():
            
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            table_df = table(pages, k)                 
            ICD.display(table_df[0])
        except: 
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))

        with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
            pickle.dump(table_df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        print('-'*80)      

,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q1,2020,23618000000,36671000000,7289000000,1771000000,69349000000,37127000000,9359000000,838000000,18811000000,2887000000,138371000000,None,None,None,829000000,712000000,835000000,12446000000,247000000,15069000000,9509000000,8489000000,33067000000,None,None,None,46688000000,544000000,59160000000,105304000000,138371000000
2,Q4,2019,19079000000,35776000000,9518000000,1852000000,66225000000,35323000000,9460000000,894000000,18715000000,2759000000,133376000000,None,None,None,1363000000,886000000,800000000,11735000000,269000000,15053000000,9524000000,7745000000,32322000000,None,None,None,45851000000,489000000,55692000000,101054000000,133376000000


All the pages from 10-Q document 0001326801-20-000048 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q3,2019,15979000000,36290000000,7673000000,2137000000,62079000000,32284000000,8403000000,853000000,18338000000,2461000000,124418000000,None,None,None,860000000,590000000,776000000,10877000000,225000000,13328000000,8356000000,8735000000,30419000000,None,None,None,45059000000,849000000,49789000000,93999000000,124418000000
2,Q4,2018,10019000000,31095000000,7587000000,1779000000,50480000000,24683000000,None,1294000000,18301000000,2576000000,97334000000,None,None,None,820000000,541000000,None,5509000000,147000000,7017000000,None,6190000000,13207000000,None,None,None,42906000000,760000000,41981000000,84127000000,97334000000


All the pages from 10-Q document 0001326801-19-000069 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Operating lease liabilities,Accrued expenses and other current liabilities,Deferred Revenue,Total current liabilities,Operating lease liabilities,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q2,2019,13877000000,34719000000,7513000000,1852000000,57961000000,29999000000,7272000000,994000000,18334000000,2446000000,117006000000,None,None,None,655000000,560000000,688000000,10878000000,198000000,12979000000,7122000000,8143000000,28244000000,None,None,None,44277000000,483000000,44968000000,88762000000,117006000000
2,Q4,2018,10019000000,31095000000,7587000000,1779000000,50480000000,24683000000,None,1294000000,18301000000,2576000000,97334000000,None,None,None,820000000,541000000,None,5509000000,147000000,7017000000,None,6190000000,13207000000,None,None,None,42906000000,760000000,41981000000,84127000000,97334000000


...

All the pages from 10-K document 0001326801-17-000007 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2015,4907000000,13527000000,2559000000,659000000,21652000000,5687000000,3246000000,18026000000,796000000,49407000000,None,None,None,196000000,217000000,1449000000,56000000,7000000,1925000000,107000000,3157000000,5189000000,None,None,None,34886000000,455000000,9787000000,44218000000,49407000000
2,None,2014,4315000000,6884000000,1678000000,513000000,13390000000,3967000000,3929000000,17981000000,699000000,39966000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2327000000,3870000000,None,None,None,30225000000,228000000,6099000000,36096000000,39966000000


All the pages from 10-K document 0001326801-16-000043 have been tableized and saved.
--------------------------------------------------------------------------------


,Quarter,Year,Cash,Marketable securities,Accounts Receivable,Prepaid expenses and other current assets,Total current assets,Property and Equipment,Intangible assets,Goodwill,Other assets,Total assets,NaN,Shareholder Equity,Current liabilities,Accounts payable,Partners payable,Accrued expenses and other current liabilities,Deferred Revenue,Current portion of capital lease obligations,Total current liabilities,Capital lease obligations,Other liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Additional paid in capital,Accumulated other comprehensive,Retained Earnings,Shareholder Equity,Total liabilities and equity
1,Q4,2014,4315000000,6884000000,1678000000,793000000,13670000000,3967000000,3929000000,17981000000,637000000,40184000000,None,None,None,176000000,202000000,866000000,66000000,114000000,1424000000,119000000,2545000000,4088000000,None,None,None,30225000000,228000000,6099000000,36096000000,40184000000
2,None,2013,3323000000,8126000000,1109000000,512000000,13070000000,2882000000,883000000,839000000,221000000,17895000000,None,None,None,87000000,181000000,555000000,38000000,239000000,1100000000,237000000,1088000000,2425000000,None,None,None,12297000000,14000000,3159000000,15470000000,17895000000


All the pages from 10-K document 0001326801-15-000006 have been tableized and saved.
--------------------------------------------------------------------------------


# Comparing the Old and the New.

In [ ]:
# Sample: Older filing style. 
import pickle
pd.set_option('display.max_columns', 100)
with open("0001564590-19-038256-10-Q.csv", "rb") as fp:
    file = pickle.load(fp)
df01 = file[0]

# df01['CSN_SUM'] = df01['Convertible Senior Notes'].values.sum()
# del df01['Convertible Senior Notes']

# df01['DeferredRevenue'] = df01['Deferred Revenue'].values.sum()
# del df01['Deferred Revenue']

# df01['NoncontrollingInterests'] = df01['Noncontrolling interests in subsidiaries'].values.sum()
# del df01['Noncontrolling interests in subsidiaries']

print(df01.shape)
df01

In [ ]:
duplicate_columns( df01 )

### Newer Filing

In [ ]:
# Sample: Newer filing style. 
import pickle 
import pandas as pd
pd.set_option('display.max_columns', 100)
with open("0001564590-20-019931-10-Q.csv", "rb") as fp:
    file = pickle.load(fp)
df02 = file[0]
print(df02.shape) 
df02

In [ ]:
checking_duplicate_columns( df02 )

In [ ]:
print('Using the Function')

duplicate_columns( df02 )


In [ ]:
# Slamming the Raw DataFrame into the dissected column cleanser. 
print('Not Using the Function')
duplicates = [k for (k,v) in Counter(df02.columns).items() if v > 1] 

for dupli in duplicates: 
    res = df02[dupli].values.sum()
    df02['Total {}'.format(dupli)] = res
    df02.drop(dupli, axis=1, inplace=True)
    
df02

In [ ]:
df02.columns

In [ ]:
checking_duplicate_columns( df2 )

# Concating dfs

### Example code: 

In [ ]:
c_ = pd.concat([df2, df01], axis=0, sort=False)

print(c_.shape)
c_

In [ ]:
# Serious Stuff. 
def duplicate_columns( DataF ):
    
    duplicates = [k for (k,v) in Counter(DataF.columns).items() if v > 1] 

    for dupli in duplicates: 
        
        res = DataF[dupli].values.sum()
        
        DataF['Serious {}'.format(dupli)] = res
        
        DataF.drop(dupli, axis=1, inplace=True)

    return DataF

In [ ]:
# Just for checking.
def checking_duplicate_columns( DataF ):
    
    duplicates = [k for (k,v) in Counter(DataF.columns).items() if v > 1] 

    
    print('Shape of df: ', DataF.shape )
    return duplicates